# **Prac 9: Modelado basado en agentes**
José Luis Haro Díaz

In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import random
import math
from matplotlib.colors import ListedColormap

In [2]:
#Parámetros
WORLD_SIZE = 51
CENTER = WORLD_SIZE // 2
POPULATION = 100
DIFFUSION_RATE = 10
EVAPORATION_RATE = 1

#Clases
class Patch:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.chemical = 0.0
        self.food = 0
        self.nest = False
        self.nest_scent = 0.0
        self.food_source_number = 0

    def distance_to(self, x2, y2):
        return math.sqrt((self.x - x2) ** 2 + (self.y - y2) ** 2)

class Turtle:
    def __init__(self, x, y, idx):
        self.x = x
        self.y = y
        self.angle = random.uniform(0, 360)
        self.carrying_food = False
        self.id = idx

    def move_forward(self):
        dx = round(math.cos(math.radians(self.angle)))
        dy = round(math.sin(math.radians(self.angle)))
        new_x = self.x + dx
        new_y = self.y + dy

        if 0 <= new_x < WORLD_SIZE and 0 <= new_y < WORLD_SIZE:
            self.x, self.y = new_x, new_y
        else:
            self.angle += 180
            self.angle %= 360

    def wiggle(self):
        self.angle += random.uniform(-40, 40)
        self.angle %= 360

class AntModel:
    def __init__(self):
        self.grid = [[Patch(x, y) for y in range(WORLD_SIZE)] for x in range(WORLD_SIZE)]
        self.turtles = [Turtle(CENTER, CENTER, i) for i in range(POPULATION)]
        self.time = 0
        self.setup_patches()

    def setup_patches(self):
        for row in self.grid:
            for patch in row:
                if patch.distance_to(CENTER, CENTER) < 5:
                    patch.nest = True
                patch.nest_scent = max(0, 200 - patch.distance_to(CENTER, CENTER))
                if patch.distance_to(int(0.8 * WORLD_SIZE), CENTER) < 5:
                    patch.food_source_number = 1
                elif patch.distance_to(int(0.2 * WORLD_SIZE), int(0.2 * WORLD_SIZE)) < 5:
                    patch.food_source_number = 2
                elif patch.distance_to(int(0.1 * WORLD_SIZE), int(0.9 * WORLD_SIZE)) < 5:
                    patch.food_source_number = 3
                if patch.food_source_number > 0:
                    patch.food = random.choice([1, 2])

    def get_patch(self, x, y):
        return self.grid[x][y]

    def uphill(self, x, y, angle, attr):
        def sample_at(a):
            dx = round(math.cos(math.radians(angle + a)))
            dy = round(math.sin(math.radians(angle + a)))
            nx = max(0, min(WORLD_SIZE - 1, x + dx))
            ny = max(0, min(WORLD_SIZE - 1, y + dy))
            return getattr(self.grid[nx][ny], attr)
        ahead = sample_at(0)
        right = sample_at(45)
        left = sample_at(-45)
        if right > ahead or left > ahead:
            return angle + 45 if right > left else angle - 45
        return angle

    def step(self):
        for ant in self.turtles:
            if ant.id > self.time:
                continue

            patch = self.get_patch(ant.x, ant.y)

            if ant.carrying_food:
                if patch.nest:
                    ant.carrying_food = False
                    ant.angle += 180
                else:
                    patch.chemical += 60
                    ant.angle = self.uphill(ant.x, ant.y, ant.angle, 'nest_scent')
            else:
                if patch.food > 0:
                    ant.carrying_food = True
                    patch.food -= 1
                    ant.angle += 180
                elif 0.05 <= patch.chemical < 2:
                    ant.angle = self.uphill(ant.x, ant.y, ant.angle, 'chemical')

            ant.wiggle()
            ant.move_forward()

        self.diffuse_and_evaporate()
        self.time += 1

    def diffuse_and_evaporate(self):
        new_chemicals = [[0.0 for _ in range(WORLD_SIZE)] for _ in range(WORLD_SIZE)]
        for x in range(WORLD_SIZE):
            for y in range(WORLD_SIZE):
                total = self.grid[x][y].chemical
                neighbors = 1
                for dx in [-1, 0, 1]:
                    for dy in [-1, 0, 1]:
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < WORLD_SIZE and 0 <= ny < WORLD_SIZE and (dx != 0 or dy != 0):
                            total += self.grid[nx][ny].chemical
                            neighbors += 1
                new_chemicals[x][y] = total * DIFFUSION_RATE / 100 / neighbors

        for x in range(WORLD_SIZE):
            for y in range(WORLD_SIZE):
                value = new_chemicals[x][y] * (100 - EVAPORATION_RATE) / 100
                self.grid[x][y].chemical = max(0.0, value)

    def get_patch_colors(self):
        colors = np.zeros((WORLD_SIZE, WORLD_SIZE, 3))  # RGB
        for x in range(WORLD_SIZE):
            for y in range(WORLD_SIZE):
                patch = self.grid[x][y]
                if patch.nest:
                    colors[x, y] = [0.5, 0, 0.5]  # Violet
                elif patch.food > 0:
                    if patch.food_source_number == 1:
                        colors[x, y] = [0, 1, 1]  # Cyan
                    elif patch.food_source_number == 2:
                        colors[x, y] = [0.5, 1, 1]  # Sky
                    elif patch.food_source_number == 3:
                        colors[x, y] = [0, 0, 1]  # Blue
                else:
                    green_intensity = min(1, patch.chemical / 5)
                    colors[x, y] = [0, green_intensity, 0]
        return colors

    def get_ant_positions(self):
        return [(ant.x, ant.y, ant.carrying_food) for ant in self.turtles]

#Visualización
model = AntModel()
fig, ax = plt.subplots()

background = ax.imshow(model.get_patch_colors(), interpolation='nearest')

scat = ax.scatter([], [], c=[], cmap=ListedColormap(['red', 'orange']), vmin=0, vmax=1, s=15)

def update(frame):
    model.step()
    background.set_data(model.get_patch_colors())
    ant_data = model.get_ant_positions()
    xs = [x for x, y, _ in ant_data]
    ys = [y for x, y, _ in ant_data]
    colors = [1 if c else 0 for _, _, c in ant_data]
    scat.set_offsets(np.c_[ys, xs])  #matplotlib usa (row, col)
    scat.set_array(np.array(colors))
    ax.set_title(f"Paso: {model.time}")
    return background, scat

from IPython.display import HTML

#Para mostrar la animacion en .ipynb se usa como embebido
ani = animation.FuncAnimation(fig, update, frames=200, interval=100, blit=False)
HTML(ani.to_jshtml())


Output hidden; open in https://colab.research.google.com to view.